In [1]:
!git clone https://github.com/LC1332/luotuo-silk-road.git
%cd  luotuo-silk-road/TuoLing
!pip install -r requirements.txt 

Cloning into 'luotuo-silk-road'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 17 (delta 3), reused 12 (delta 2), pack-reused 0
Unpacking objects: 100% (17/17), 12.99 MiB | 4.46 MiB/s, done.
/content/luotuo-silk-road/TuoLing
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-eijacg7d
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-eijacg7d
  Resolved https://github.com/huggingface/peft.git to commit 098962fa6515f2e4fe83a757f5995d3ffbb1c373
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.3/76.3 MB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.8/

In [2]:
from modeling_chatglm import ChatGLMForConditionalGeneration
import torch


torch.set_default_tensor_type(torch.cuda.HalfTensor)
model = ChatGLMForConditionalGeneration.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True, device_map='auto')

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


In [4]:
from peft import get_peft_model, LoraConfig, TaskType

peft_path = "output/chatglm-lora.pt"

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, inference_mode=True,
    r=8,
    lora_alpha=32, lora_dropout=0.1
)

model = get_peft_model(model, peft_config)
model.load_state_dict(torch.load(peft_path), strict=False)
torch.set_default_tensor_type(torch.cuda.FloatTensor)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/lib64-nvidia did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--listen_host=172.28.0.12 --target_host=172.28.0.12 --tunnel_background_save_url=https'), PosixPath('//colab.research.google.com/tun/m/cc48301118ce562b961b3c22d803539adc1e0c19/gpu-a100-s-6dkcgffe5bbz --tunnel_background_save_delay=10s --tunnel_periodic_background_save_frequency=30m0s --enable_output_coalescing=true --output_coalescing_required

In [13]:
def evaluate(instruction, input=None):
  with torch.no_grad():
    feature = format_example({'instruction': f'{instruction}', 'output': '', 'input': ''})
    input_text = feature['context']
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    out = model.generate(
            input_ids=input_ids,
            max_length=150,
            temperature=0
        )
    answer = tokenizer.decode(out[0])
    print(answer)

In [14]:
evaluate(input("Instruction: "))

Instruction: 谁创造了你?
Instruction: 谁创造了你?
Answer: 我是由商汤科技的李鲁鲁和冷子昂,以及华中师范大学的陈启源共同开发的。


In [15]:
evaluate(input("Instruction: "))

Instruction: 你最喜欢的电影是什么？
Instruction: 你最喜欢的电影是什么?
Answer: 我最喜欢的电影是《疯狂动物城》(Zootopia),这是一部动画喜剧电影,讲述了一只狐狸和一只兔子之间的友谊和冒险故事。这部电影故事情节有趣,动画效果出色,让我非常喜欢它。
